# 3.3 A simple text classifier

## Data loading and preprocessing

In [2]:
import pandas as pd
from google.colab import files
#uploaded = files.upload()
data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', 
                   encoding='latin-1',
                   header=None)
data = data.sample(frac=1) 
data.head()
#print(data[5][0])

,0,1,2,3,4,5
436841,0,2065774730,Sun Jun 07 09:15:51 PDT 2009,NO_QUERY,bradleycfox,way too many cute dogs for me this morning
1282361,4,2001829949,Tue Jun 02 02:13:07 PDT 2009,NO_QUERY,nicblondel,Bored @school...cant w8 29 days and i'm in the...
1306390,4,2011655373,Tue Jun 02 19:21:34 PDT 2009,NO_QUERY,TimBaileyGR,Looking forward Guy Kawasaki's âStart Someth...
253621,0,1984033076,Sun May 31 14:18:37 PDT 2009,NO_QUERY,PrettyGreyx3,@day26keke http://twitpic.com/6c2if - owww yes...
1276226,4,2001180449,Tue Jun 02 00:04:47 PDT 2009,NO_QUERY,lavasusan,@dreabean13 Congratulations on the new baby! N...


In [0]:
# train-test split
train_size = int(0.7*len(data))
features = data[5]
targets = data[0]
X_train, X_test = features.values[:train_size], features.values[train_size:]
y_train, y_test = targets.values[:train_size], targets.values[train_size:]

y_train[y_train==2] = 1
y_train[y_train==4] = 2

y_test[y_test==2] = 1
y_test[y_test==4] = 2

In [0]:
# count vectorize them
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(max_features=100)
X_train_num = count_vectorizer.fit_transform(X_train).toarray()
X_test_num = count_vectorizer.transform(X_test).toarray()

In [23]:
instance = 46544
print(X_train[instance])
print(X_train_num[instance])

I ran outta bathroom passes.  maybe mrs. French will be nice today?
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]


## Text classification

In [22]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.leaky_relu), # x if x > 0 else alpha*x 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3)
])

adam_optimizer = tf.keras.optimizers.Adam()
model.compile(
    optimizer=adam_optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[
      'accuracy',
    ]
)

print(y_train)
model.fit(X_train_num, 
          y_train,
          batch_size=64,
          epochs=2,
          validation_split=0.1,
          verbose=1)

[0 2 2 ... 2 0 0]
Train on 1008000 samples, validate on 112000 samples
Epoch 1/2
1008000/1008000 [==============================] - 43s 43us/sample - loss: 0.6168 - accuracy: 0.6578 - val_loss: 0.6042 - val_accuracy: 0.6704
Epoch 2/2
1008000/1008000 [==============================] - 43s 43us/sample - loss: 0.6095 - accuracy: 0.6638 - val_loss: 0.6022 - val_accuracy: 0.6719


In [0]:
!pip install --upgrade tensorflow